In [ ]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
df = pd.read_csv("billboard100_data.csv")

In [ ]:
df['title'] = df['title'].map(str.lower) # converting the entries to lower
df['author'] = df['author'].map(str.lower) # converting the entries to lower
df

In [ ]:
#from fuzzywuzzy import process
#def check_mistakes(song_name, reference_database, title_column, artist_column):
#    best_match, score = process.extractOne(song_name, reference_database[title_column].values)
#    if score >= 70:
#        artist = reference_database[reference_database[title_column] == best_match][artist_column][0]
#        answer = input("f\Did you mean {best_match} by {artist}? Y/N: ")
#        #answer = input(“”).lower().title()
#        if answer == "Y":
#            return best_match
#        if answer == "N":
#            pass
#    else:
#        print("error")

In [ ]:
x = input('Enter a song you like: ')
x_artist = df.loc[df['title'].map(str.lower)  == x.lower(), 'author'] # artist corresponding to song x
artist_question = input('Did you mean ' + x.lower().title() + ' by ' + x_artist.to_string(index=False).title() + '? (y/n)')
#x_artist.to_string(index=False).title() converts the series x_artist to a string and then uppers the first alphabets
#x.lower().title()  lowers the entries and then uppers the first alphabets
if artist_question == 'y':
    if x.lower() in df.values:   # converts te entered x into all lower case
        rec_song = df['title'].sample().to_string(index=False) # recommendation song. I just removed the indexes as they are not required as the output  
        rec_artist = df.loc[df['title'] == rec_song, 'author'].to_string(index=False) # artist of the recommended song
        # to_string(index=False) lets you output without index
        if x != rec_song:
            print("You might like this song: " + rec_song.title() + " by " + rec_artist.title())  # .title() makes the first alphabet in every word to upper   
    else:
        print("\nThis song does not exists in the data set.")
else:
    print("\nThis song does not exists in the data set.")
        

# Spotify dataframe

In [ ]:
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))


Playlists

In [ ]:
# Rock classics playlist link: https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=013db49871054a73   
# Rock classics playlist ID: 37i9dQZF1DWXRqgorJj26U 
rock_playlist_id = "37i9dQZF1DWXRqgorJj26U"
playlist_rock = sp.user_playlist_tracks("spotify", rock_playlist_id)
#import pprint
#pprint.pprint(playlist_rock) # in case you want to check the playlist

jazz_playlist_id = "37i9dQZF1DXbITWG1ZJKYt"
pop_playlist_id = "7iUGicmg9AkkFhkLiYqYpA"
blues_playlist_id = "37i9dQZF1DXd9rSDyQguIk"
techno_playlist_id = "18vUeZ9BdtMRNV6gI8RnR6"
allgenre_playlist_id = "52oPdG5WK1KcMf78FvFXIa"

electronic_playlist_id = "37i9dQZF1DX0wMD4IoQ5aJ"
house_playlist_id = "2otQLmbi8QWHjDfq3eL0DC"    
trend23_playlist_id = "5JNNpb0q7ZIM3s3j355Anr"    
afro_playlist_id = "37i9dQZF1DWYs2pvwxWA7l"
rap_playlist_id = "4riovLwMCrY3q0Cd4e0Sqp"    
brazil_playlist_id = "37i9dQZF1DX9FSYmdU9mbT"
spanish_playlist_id = "0dexh2j27EfJtsncx1vjhW"
kpop_playlist_id = "14dxceLGBZkRAj6WJTfkpj"    


playlist_ids = [rock_playlist_id, jazz_playlist_id, pop_playlist_id, blues_playlist_id, 
                techno_playlist_id, allgenre_playlist_id, electronic_playlist_id, house_playlist_id,
               trend23_playlist_id, afro_playlist_id, rap_playlist_id, brazil_playlist_id, 
               spanish_playlist_id, kpop_playlist_id]

In [ ]:
# Extracting many songs from the playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
# Retrieve all tracks from the playlists
import time
tracks = []
counter = 0
for playlist_id in playlist_ids:
    tracks += get_playlist_tracks('spotify', playlist_id)
    counter += 1
    if counter % 50 == 0:
        time.sleep(5) # sleep for 5 seconds every 50 playlists

In [ ]:
# extract song ids from the playlist
list_of_audio_features=[]
counter1 = 0
for item in range(0,len(tracks)):
    list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])
    counter1 += 1
    if counter % 50 == 0:
        time.sleep(5) # sleep for 5 seconds every 50 playlists

In [ ]:
# creating the data frame
df_spotify=pd.DataFrame(list_of_audio_features)    
df_spotify=df_spotify[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df_spotify

In [ ]:
df_spotify.to_csv('spotify_data.csv', index=False)